# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the most widely followed stock market index in the world. The largest fund that tracks this index is the SPDR® S&P 500® ETF Trust, which has over $250 billion in assets under management.

The goal of this project is to create a Python script that will take the value of your portfolio and calculate how many shares of each S&P 500 constituent you should buy in order to create an equal-weight version of the index fund. This notebook is based on the freeCodeCamp course "Algorithmic Trading Using Python - Full Course" which can be accessed at the following URL:

https://www.youtube.com/watch?v=xfzGZB4HhEE

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [18]:
# If you want to use this script, you need to install the following libraries
# pip install numpy pandas yfinance

import numpy as np
import pandas as pd
import yfinance as yf
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

However, accessing the index provider's API requires a paid subscription, which is outside the scope of this project.

To proceed, you have two options:

1. **Run the extractor script**: If you want to obtain the most up-to-date list of S&P 500 tickers, you can run the `sp500_tickers_extractor.py` script. This script scrapes the data from Wikipedia and generates the latest `.parquet` file with the tickers [Click this link to download the script](https://github.com/Sebas20031-DS/Algoritmic-Trading-Projects/blob/main/S%26P%20500%20equal%20weight/sp500_tickers_extractor.ipynb).

2. **Use the static version**: If you prefer to skip the scraping and work with pre-collected data, you can download the static `.parquet` file from the repository. The file will be available on GitHub. Simply download it and use it directly in your notebook (take into account that it may be outdated). [Click this link to download the static .parquet file](https://github.com/Sebas20031-DS/Algoritmic-Trading-Projects/blob/main/S%26P%20500%20equal%20weight/sp500_tickers.parquet)


Once extracted or downloaded, move it to your project path so you can load the .parquet file into your Jupyter Notebook like this: 

In [2]:
stocks = pd.read_parquet('sp500_tickers.parquet')

## Using yfinance to Fetch Stock Data

This script demonstrates how to use the yfinance library to fetch the latest stock price and market capitalization for a given stock symbol.


In [3]:
symbol = 'AAPL'
stock = yf.Ticker(symbol)
latest_price = stock.history(period="1d")["Close"].iloc[-1]
market_cap = stock.info["marketCap"] # Get market capitalization

print(f"Latest Price: {latest_price}")
print(f"Market Cap: {market_cap}")

Latest Price: 213.49000549316406
Market Cap: 3207068385280


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [4]:
my_columns = ['Ticker', 'Price', 'Market Capitalization']
dataframe = pd.DataFrame(columns=my_columns)

In [ ]:
# Create a new row as a DataFrame
new_row = pd.DataFrame([[symbol, latest_price, market_cap]], columns=my_columns)

# The next line of code will give you a FutureWarning message (pandas version used in this project is 2.2.3)
# you can check yours with print(pd.__version__)
# Use pd.concat to add the new row
dataframe = pd.concat([dataframe, new_row], ignore_index=True)

dataframe

C:\Users\sebas\AppData\Local\Temp\ipykernel_161608\1316518063.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_row], ignore_index=True)


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

**Warning:** While the approach works well for most stocks, there may be some tickers that do not return data due to various reasons, such as the stock being temporarily unavailable or incorrect ticker symbols. In such cases, we recommend implementing an exception handling mechanism to catch and manage these errors.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

# Iterate over each stock in the list
for stock in stocks['Ticker']:
    try:
        # Get ticker data
        Ticker = yf.Ticker(stock)

        # Get the latest price
        history = Ticker.history(period="1d")
        if history.empty:  # If there is no data, skip to the next stock
            print(f"Warning: No data found for {stock}. Skipping...")
            continue
        latest_price = history["Close"].iloc[-1]

        # Get market capitalization
        market_cap = Ticker.info.get("marketCap", "N/A")  # Use 'N/A' if no data is available

        # Create a new DataFrame with the new row
        new_row = pd.DataFrame([[stock, latest_price, market_cap]], columns=my_columns)

        # Concatenate the new row without future compatibility issues
        final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)

    except Exception as e:
        print(f"Error processing {stock}: {e}")  # Print error message but continue

C:\Users\sebas\AppData\Local\Temp\ipykernel_161608\1388065216.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
$BF.B: possibly delisted; no price data found  (period=1d)


$BRK.B: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [7]:
final_dataframe

,Ticker,Price,Market Capitalization
0,A,121.180000,34548781056
1,AAPL,213.490005,3207068385280
2,ABBV,211.770004,373848178688
3,ABNB,122.860001,76337840128
4,ABT,126.709999,219755692032
...,...,...,...
496,XYL,122.760002,29836326912
497,YUM,155.050003,43274768384
498,ZBH,109.970001,21890959360
499,ZBRA,278.970001,14333255680


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we still have not calculated the weight of each stock and the number of shares of each stock to buy.

### **How the S&P 500 Weighs Stocks**

The **S&P 500 Index** is a **market-capitalization-weighted index**, meaning that the weight of each stock is determined by its **market capitalization** (the total value of a company's outstanding shares). This ensures that larger companies have a greater influence on the index performance.

#### **1. Calculating Each Stock's Market Cap**
Each stock's **market capitalization** is calculated as:

$$
\text{Market Cap} = \text{Stock Price} \times \text{Shares Outstanding}
$$

Where:
- **Stock Price**: The latest price of the stock.
- **Shares Outstanding**: The total number of shares issued by the company.

#### **2. Summing Up All Market Caps**
The total market capitalization of the S&P 500 is the sum of all individual market caps:

$$
\text{Total Market Cap} = \sum_{i=1}^{500} \text{Market Cap}_i
$$

#### **3. Calculating Each Stock’s Weight in the Index**
Each stock's weight in the index is determined by:

$$
\text{Weight} = \frac{\text{Stock Market Cap}}{\text{Total Market Cap}}
$$

This means that companies with **higher market capitalization have a greater impact** on the index.

#### **4. Determining Position Size in a Portfolio**
If an investor wants to replicate the S&P 500 using a **personal portfolio**, they need to allocate funds to each stock based on its weight. Given a **portfolio size (P)**, the **position size** for each stock is:

$$
\text{Position Size} = \text{Weight} \times P
$$

This determines how much money should be invested in each stock.

#### **5. Calculating the Number of Shares to Buy**
Once the **position size** is known, the **number of shares to buy** is:

$$
\text{Shares to Buy} = \frac{\text{Position Size}}{\text{Stock Price}}
$$

Since shares cannot be fractional in most cases, the number of shares is typically rounded down to the nearest whole number.


In [32]:
# Calculate total market capitalization (sum of all individual market caps)
total_market_cap = final_dataframe["Market Capitalization"].sum()

# Calculate weight of each stock in the index
final_dataframe["Weight"] = final_dataframe["Market Capitalization"] / total_market_cap

# Get portfolio size from user
while True:
    portafolio_size = input("Enter the value of your portfolio: ")
    try:
        val = float(portafolio_size)
        if val > 0:
            break
        else:
            print("Please enter a positive number greater than 0.")
    except ValueError:
        print("That's not a number! Please try again.")

# Calculate the dollar amount to allocate to each stock
final_dataframe["Position Size ($)"] = final_dataframe["Weight"] * val

# Calculate the number of shares to buy for each stock
final_dataframe["Shares to Buy"] = final_dataframe["Position Size ($)"] / final_dataframe["Price"]
final_dataframe["Shares to Buy (Round)"] = final_dataframe["Shares to Buy"].astype(int)  # Convert to whole shares

final_dataframe

,Ticker,Price,Market Capitalization,Weight,Position Size ($),Shares to Buy,Shares to Buy (Round)
0,A,121.180000,34548781056,0.00067,670.473999,5.532877,5
1,AAPL,213.490005,3207068385280,0.062238,62238.258462,291.527738,291
2,ABBV,211.770004,373848178688,0.007255,7255.118,34.259422,34
3,ABNB,122.860001,76337840128,0.001481,1481.457098,12.058091,12
4,ABT,126.709999,219755692032,0.004265,4264.708424,33.657237,33
...,...,...,...,...,...,...,...
496,XYL,122.760002,29836326912,0.000579,579.021338,4.716694,4
497,YUM,155.050003,43274768384,0.00084,839.81565,5.416418,5
498,ZBH,109.970001,21890959360,0.000425,424.828854,3.863134,3
499,ZBRA,278.970001,14333255680,0.000278,278.159604,0.997095,0


## **Saving the Final DataFrame to an Excel File with Formatting**

After calculating the recommended trades, the next step is to **save the results in an Excel file** for better readability and record-keeping.

In [33]:
# Define the filename dynamically based on the portfolio size
filename = f"Recommended_Trades_for_Portfolio_{int(val)}.xlsx"

# Save the DataFrame to an Excel file
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    final_dataframe.to_excel(writer, sheet_name="Trades", index=False)

    # Access the workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets["Trades"]

    # Formatting: Adjust column width and apply styles
    for col in worksheet.columns:
        max_length = 0
        col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)

        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))  # Find max length in column
            except:
                pass

        worksheet.column_dimensions[col_letter].width = max_length + 2  # Adjust column width

    # Apply header styling
    for cell in worksheet[1]:  # First row contains headers
        cell.font = Font(bold=True, color="FFFFFF")  # Bold white text
        cell.alignment = Alignment(horizontal="center")  # Center text
        cell.fill = openpyxl.styles.PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")  # Blue background

print(f"Excel file '{filename}' saved successfully!")

Excel file 'Recommended_Trades_for_Portfolio_1000000.xlsx' saved successfully!
